StyleGAN3

In [ ]:
#@markdown ###Check GPU type
#@markdown Factory reset runtime if you don't have the desired GPU.
#@markdown - V100 = Excellent (*Available only for Colab Pro users*)
#@markdown - P100 = Very Good
#@markdown - T4 = Good (*preferred*)
#@markdown - K80 = Meh
#@markdown - P4 = (*Not Recommended*)

!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-6aebdffa-906a-0659-7cc2-1f760f176747)


In [ ]:

#!pip install --upgrade torch==1.9.1+cu111 torchvision==0.10.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install --upgrade https://download.pytorch.org/whl/nightly/cu111/torch-1.11.0.dev20211012%2Bcu111-cp37-cp37m-linux_x86_64.whl https://download.pytorch.org/whl/nightly/cu111/torchvision-0.12.0.dev20211012%2Bcu111-cp37-cp37m-linux_x86_64.whl
!pip install --upgrade torch==1.11.0+cu111 torchvision==0.12.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

import sys
import io
import os, time, glob
import pickle
import shutil
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import clip
import unicodedata
import re
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from IPython.display import display
from einops import rearrange
from google.colab import files

from IPython.display import clear_output
clear_output()

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)


In [ ]:
NAME = 'stylegan3'
GDRIVE_MOUNT = '/content/drive'
BASE_DIR = f'{GDRIVE_MOUNT}/MyDrive/colab/'
CODE_DIR = f'{BASE_DIR}{NAME}/'

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount(GDRIVE_MOUNT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
if os.path.isdir(CODE_DIR):
    %cd {CODE_DIR}
    !git pull
else:
    !git clone https://github.com/ManuelKugelmann/stylegan3-fun.git {CODE_DIR}
    %cd {CODE_DIR}

/content/drive/MyDrive/colab/stylegan3
Already up to date.


In [ ]:
#@title Generate an image
#@markdown StyleGAN3 pre-trained models for config T (translation equiv.) and config R (translation and rotation equiv.)
seed = 4011 #@param {type:"slider", min:0, max:9999, step:1}

baselink ='https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/'
model = "stylegan3-r-afhqv2-512x512.pkl" #@param ["stylegan3-r-afhqv2-512x512.pkl", "stylegan3-r-ffhq-1024x1024.pkl", "stylegan3-r-ffhqu-1024x1024.pkl","stylegan3-r-ffhqu-256x256.pkl","stylegan3-r-metfaces-1024x1024.pkl","stylegan3-r-metfacesu-1024x1024.pkl","stylegan3-t-afhqv2-512x512.pkl","stylegan3-t-ffhq-1024x1024.pkl","stylegan3-t-ffhqu-1024x1024.pkl","stylegan3-t-ffhqu-256x256.pkl","stylegan3-t-metfaces-1024x1024.pkl","stylegan3-t-metfacesu-1024x1024.pkl"]

# Generate an image using pre-trained model 
!python gen_images.py --outdir=out --trunc=1 \
 --seeds=$seed --network=$baselink$model
 
import matplotlib.pyplot as plt
from PIL import Image

plt.figure(figsize=(10,10))
img = Image.open('/content/stylegan3/out/seed%04d.png' % seed);
plt.imshow(img);
plt.axis('off');

In [ ]:
#@markdown #**Model selection** 🎭
#@markdown - FFHQ: Trained with human faces.
#@markdown - MetFaces: Trained with paintings/portraits of human faces.
#@markdown - AFHQv2: Trained with animal faces.
#@markdown - Cosplay: Trained by [l4rz](https://twitter.com/l4rz) with cosplayer's faces.
#@markdown - Wikiart: Trained by [Justin Pinkney](https://www.justinpinkney.com/) with the Wikiart 1024 dataset.
#@markdown - Landscapes: Trained by [Justin Pinkney](https://www.justinpinkney.com/) with the LHQ dataset.

#@markdown **Run this cell again if you change the model**.

#@markdown ---

base_url = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/"

Model = 'FFHQ' #@param ["FFHQ", "MetFaces", "AFHQv2", "cosplay", "Wikiart", "Landscapes"]

#@markdown ---

model_name = {
    "FFHQ": base_url + "stylegan3-t-ffhqu-1024x1024.pkl",
    "MetFaces": base_url + "stylegan3-r-metfacesu-1024x1024.pkl",
    "AFHQv2": base_url + "stylegan3-t-afhqv2-512x512.pkl",
    "cosplay": "https://l4rz.net/cosplayface-snapshot-stylegan3t-008000.pkl",
    "Wikiart": "https://drive.google.com/u/0/open?id=18MOpwTMJsl_Z17q-wQVnaRLCUFZYSNkj",
    "Landscapes": "https://drive.google.com/u/0/open?id=14UGDDOusZ9TMb-pOrF0PAjMGVWLSAii1"
}

network_url = model_name[Model]

with open(fetch_model(network_url), 'rb') as fp:
  G = pickle.load(fp)['G_ema'].to(device)

zs = torch.randn([10000, G.mapping.z_dim], device=device)
w_stds = G.mapping(zs, None).std(0)